In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install -q datasets --no-deps

In [3]:
from datasets import load_dataset

dataset = load_dataset("JacobLinCool/VoiceBank-DEMAND-16k")

In [4]:
train = dataset['train']
test = dataset['test']

In [5]:
print(train.shape)

(11572, 3)


Each row is a dictionary. The clean and noisy data are in the form of numpy array which should be changed to tensor.
Also, the audio are already downsampled to a sampling rate of 16k

In [6]:
print(train[0])

{'id': 'p226_001', 'clean': {'path': 'p226_001.wav', 'array': array([-0.00286865, -0.00497437, -0.00469971, ..., -0.00201416,
       -0.00152588, -0.00021362]), 'sampling_rate': 16000}, 'noisy': {'path': 'p226_001.wav', 'array': array([-0.00283813, -0.00488281, -0.00473022, ...,  0.006073  ,
        0.00564575,  0.00540161]), 'sampling_rate': 16000}}


In [7]:
import os
import tensorflow as tf

def stft(row, n_fft=510, hop_length=128, win_length=510):
    clean_wave = tf.convert_to_tensor(row['clean']['array'], dtype=tf.float32)
    noisy_wave = tf.convert_to_tensor(row['noisy']['array'], dtype=tf.float32)

    clean_stft = tf.signal.stft(clean_wave, fft_length = n_fft, frame_length = win_length, 
                                frame_step = hop_length, window_fn=tf.signal.hann_window)
    noisy_stft = tf.signal.stft(noisy_wave, fft_length = n_fft, frame_length = win_length, 
                                frame_step = hop_length, window_fn=tf.signal.hann_window)

    clean_tensor = tf.stack([tf.math.real(clean_stft), tf.math.imag(clean_stft)], axis=-1)
    noisy_tensor = tf.stack([tf.math.real(noisy_stft), tf.math.imag(noisy_stft)], axis=-1)

    return clean_tensor, noisy_tensor

2025-06-21 17:56:41.945049: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750528602.158218     152 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750528602.215067     152 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [9]:
clean_tensor, noisy_tensor = stft(train[0])
print(clean_tensor.shape)

(282, 256, 2)


In [10]:
new_dataset = []

for i in range(train.shape[0]):
    row = train[i]
    clean_tensor, noisy_tensor = stft(row)
    new_dataset.append({
        'noisy_tensor': noisy_tensor,
        'clean_tensor': clean_tensor
    })